Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [5]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [6]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [7]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [8]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [11]:
def weight_variable(shape):
    return tf.Variable(tf.truncated_normal(dtype = tf.float32, shape = shape))
def bias_variable(shape):
    return tf.Variable(tf.zeros(shape))

batch_size = 128
hidden_nodes = 1024
lambda_ = 0.001

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    x = tf.placeholder(tf.float32, shape=(None, image_size * image_size))
    y = tf.placeholder(tf.float32, shape=(None, num_labels))
    
    # Variables.
    weights_1 = weight_variable([image_size * image_size, hidden_nodes])
    biases_1 = bias_variable([hidden_nodes])
  
    hidden_inputs = tf.matmul(x, weights_1) + biases_1
    hidden_outputs = tf.nn.relu(hidden_inputs)
    
    weights_2 = weight_variable([hidden_nodes, num_labels])
    biases_2 = bias_variable([num_labels])
    
    logits = tf.matmul(hidden_outputs, weights_2)+biases_2
    
  
    
    
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=logits))\
        + lambda_*tf.nn.l2_loss(weights_1) + lambda_*tf.nn.l2_loss(weights_2)
  
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

    prediction = tf.nn.softmax(logits)
    accuracy = 100*tf.reduce_mean(tf.cast(tf.equal(tf.arg_max(prediction, 1), tf.arg_max(y, 1)), dtype = tf.float32))

In [13]:
num_steps = 20001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {x : batch_data, y : batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % session.run(accuracy, feed_dict = {x:batch_data, y: batch_labels}))
            print("Validation accuracy: %.1f%%" % session.run(accuracy, feed_dict = {x:valid_dataset, y: valid_labels}))
    print("Test accuracy: %.1f%%" % session.run(accuracy, feed_dict = {x:test_dataset, y: test_labels}))

Initialized
Minibatch loss at step 0: 636.731506
Minibatch accuracy: 34.4%
Validation accuracy: 26.3%
Minibatch loss at step 500: 296.773041
Minibatch accuracy: 91.4%
Validation accuracy: 79.9%
Minibatch loss at step 1000: 268.788513
Minibatch accuracy: 82.0%
Validation accuracy: 81.4%
Minibatch loss at step 1500: 238.466888
Minibatch accuracy: 89.1%
Validation accuracy: 79.4%
Minibatch loss at step 2000: 214.282242
Minibatch accuracy: 93.8%
Validation accuracy: 82.8%
Minibatch loss at step 2500: 193.218643
Minibatch accuracy: 89.1%
Validation accuracy: 83.9%
Minibatch loss at step 3000: 173.747025
Minibatch accuracy: 89.8%
Validation accuracy: 83.2%
Minibatch loss at step 3500: 156.260361
Minibatch accuracy: 93.0%
Validation accuracy: 83.8%
Minibatch loss at step 4000: 140.229034
Minibatch accuracy: 92.2%
Validation accuracy: 82.1%
Minibatch loss at step 4500: 126.758286
Minibatch accuracy: 92.2%
Validation accuracy: 83.5%
Minibatch loss at step 5000: 115.219849
Minibatch accuracy: 91

Validation accuracy: 88.8%
    
Test accuracy: 94.7%

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [15]:
num_steps = 10000
batch_size = 128

mini_train_dataset = train_dataset[:500,:]
mini_train_labels = train_labels[:500]

with tf.Session(graph = graph) as session:
    tf.global_variables_initializer().run()
    for step in range(num_steps):
        offset = (step * batch_size) % (mini_train_labels.shape[0] - batch_size)
        batch_data = mini_train_dataset[offset:(offset + batch_size), :]
        batch_labels = mini_train_labels[offset:(offset + batch_size), :]
        
        feed_dict = {x : batch_data, y : batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, prediction], feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % session.run(accuracy, feed_dict = {x:batch_data, y: batch_labels}))
            print("Validation accuracy: %.1f%%" % session.run(accuracy, feed_dict = {x:valid_dataset, y: valid_labels}))
    print("Test accuracy: %.1f%%" % session.run(accuracy, feed_dict = {x:test_dataset, y: test_labels}))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Minibatch loss at step 0: 612.612915
Minibatch accuracy: 35.9%
Validation accuracy: 24.7%
Minibatch loss at step 500: 284.089752
Minibatch accuracy: 100.0%
Validation accuracy: 70.2%
Minibatch loss at step 1000: 257.054138
Minibatch accuracy: 100.0%
Validation accuracy: 70.2%
Minibatch loss at step 1500: 232.590591
Minibatch accuracy: 100.0%
Validation accuracy: 70.2%
Minibatch loss at step 2000: 210.455826
Minibatch accuracy: 100.0%
Validation accuracy: 70.2%
Minibatch loss at step 2500: 190.427048
Minibatch accuracy: 100.0%
Validation accuracy: 70.2%
Minibatch loss at step 3000: 172.304977
Minibatch accuracy: 100.0%
Validation accuracy: 70.2%
Minibatch loss at step 3500: 155.907089
Minibatch accuracy: 100.0%
Validation accuracy: 70.2%
Minibatch loss at step 4000: 141.069794
Minibatch accuracy: 100.0%
Validation accuracy: 70.2%
Minibatch loss at step 4500: 127.644623
Minibatch accuracy: 100.0%
Validation accurac

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [26]:
graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    x = tf.placeholder(tf.float32, shape=(None, image_size * image_size))
    x_dropout = tf.nn.dropout(x, 0.5)
    y = tf.placeholder(tf.float32, shape=(None, num_labels))
    
    # Variables.
    weights_1 = weight_variable([image_size * image_size, hidden_nodes])
    biases_1 = bias_variable([hidden_nodes])
  
    hidden_outputs = tf.nn.relu(tf.matmul(x, weights_1) + biases_1)
    hidden_outputs_dropout = tf.nn.dropout(tf.nn.relu(tf.matmul(x_dropout, weights_1) + biases_1), 0.5)
    weights_2 = weight_variable([hidden_nodes, num_labels])
    biases_2 = bias_variable([num_labels])
    
    logits = tf.matmul(hidden_outputs, weights_2)+biases_2
    logits_dropout = tf.matmul(hidden_outputs_dropout, weights_2)+biases_2
    
  
    
    
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=logits_dropout))\
        + lambda_*tf.nn.l2_loss(weights_1) + lambda_*tf.nn.l2_loss(weights_2)
  
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    prediction = tf.nn.softmax(logits)
    accuracy = 100*tf.reduce_mean(tf.cast(tf.equal(tf.arg_max(prediction, 1), tf.arg_max(y, 1)), dtype = tf.float32))

In [28]:
num_steps = 20001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {x : batch_data, y : batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % session.run(accuracy, feed_dict = {x:batch_data, y: batch_labels}))
            print("Validation accuracy: %.1f%%" % session.run(accuracy, feed_dict = {x:valid_dataset, y: valid_labels}))
    print("Test accuracy: %.1f%%" % session.run(accuracy, feed_dict = {x:test_dataset, y: test_labels}))

Initialized
Minibatch loss at step 0: 1033.608643
Minibatch accuracy: 22.7%
Validation accuracy: 15.1%
Minibatch loss at step 500: 288.601379
Minibatch accuracy: 84.4%
Validation accuracy: 76.9%
Minibatch loss at step 1000: 154.120483
Minibatch accuracy: 69.5%
Validation accuracy: 74.8%
Minibatch loss at step 1500: 83.966011
Minibatch accuracy: 71.9%
Validation accuracy: 71.2%
Minibatch loss at step 2000: 43.626434
Minibatch accuracy: 79.7%
Validation accuracy: 75.1%
Minibatch loss at step 2500: 26.018332
Minibatch accuracy: 82.8%
Validation accuracy: 77.8%
Minibatch loss at step 3000: 15.886987
Minibatch accuracy: 77.3%
Validation accuracy: 79.5%
Minibatch loss at step 3500: 9.721693
Minibatch accuracy: 83.6%
Validation accuracy: 80.1%
Minibatch loss at step 4000: 6.116566
Minibatch accuracy: 87.5%
Validation accuracy: 81.6%
Minibatch loss at step 4500: 3.990018
Minibatch accuracy: 85.2%
Validation accuracy: 82.0%
Minibatch loss at step 5000: 2.698494
Minibatch accuracy: 84.4%
Validat

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [42]:
hidden_nodes_1 = 1024
hidden_nodes_2 = 512
graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    x = tf.placeholder(tf.float32, shape=(None, image_size * image_size))
    x_dropout = tf.nn.dropout(x, 0.5)
    y = tf.placeholder(tf.float32, shape=(None, num_labels))
    
    # Variables.
    weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes_1], stddev=np.sqrt(1.0 / (image_size * image_size))))
    biases_1 = bias_variable([hidden_nodes_1])
  
    hidden_outputs_1 = tf.nn.relu(tf.matmul(x, weights_1) + biases_1)
    hidden_outputs_dropout_1 = tf.nn.dropout(tf.nn.relu(tf.matmul(x_dropout, weights_1) + biases_1), 0.5)
    
    weights_2 = tf.Variable(tf.truncated_normal([hidden_nodes_1, hidden_nodes_2], stddev=np.sqrt(1.0 / (hidden_nodes_1))))
    biases_2 = bias_variable([hidden_nodes_2])
    
    hidden_outputs_2 = tf.nn.relu(tf.matmul(hidden_outputs_1, weights_2) + biases_2)
    hidden_outputs_dropout_2 = tf.nn.dropout(tf.nn.relu(tf.matmul(hidden_outputs_dropout_1, weights_2) + biases_2), 0.5)
    
    weights_3 = tf.Variable(tf.truncated_normal([hidden_nodes_2, num_labels], stddev=np.sqrt(1.0 / (hidden_nodes_2))))
    biases_3 = bias_variable([num_labels])
    
    
    logits = tf.matmul(hidden_outputs_2, weights_3)+biases_3
    logits_dropout = tf.matmul(hidden_outputs_dropout_2, weights_3)+biases_3
    
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=logits_dropout))\
        + lambda_*tf.nn.l2_loss(weights_1) + lambda_*tf.nn.l2_loss(weights_2) + lambda_*tf.nn.l2_loss(weights_3)
  
    # Optimizer.
    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.1, global_step, 20000, 0.80)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

    prediction = tf.nn.softmax(logits)
    accuracy = 100*tf.reduce_mean(tf.cast(tf.equal(tf.arg_max(prediction, 1), tf.arg_max(y, 1)), dtype = tf.float32))

In [43]:
num_steps = 20001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {x : batch_data, y : batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % session.run(accuracy, feed_dict = {x:batch_data, y: batch_labels}))
            print("Validation accuracy: %.1f%%" % session.run(accuracy, feed_dict = {x:valid_dataset, y: valid_labels}))
    print("Test accuracy: %.1f%%" % session.run(accuracy, feed_dict = {x:test_dataset, y: test_labels}))

Initialized
Minibatch loss at step 0: 3.041848
Minibatch accuracy: 11.7%
Validation accuracy: 10.0%
Minibatch loss at step 500: 1.187704
Minibatch accuracy: 86.7%
Validation accuracy: 82.0%
Minibatch loss at step 1000: 1.167134
Minibatch accuracy: 81.2%
Validation accuracy: 82.8%
Minibatch loss at step 1500: 1.045249
Minibatch accuracy: 84.4%
Validation accuracy: 83.7%
Minibatch loss at step 2000: 0.861131
Minibatch accuracy: 90.6%
Validation accuracy: 84.2%
Minibatch loss at step 2500: 0.883298
Minibatch accuracy: 85.9%
Validation accuracy: 84.7%
Minibatch loss at step 3000: 0.978139
Minibatch accuracy: 87.5%
Validation accuracy: 84.7%
Minibatch loss at step 3500: 0.887636
Minibatch accuracy: 88.3%
Validation accuracy: 85.0%
Minibatch loss at step 4000: 0.826383
Minibatch accuracy: 90.6%
Validation accuracy: 85.3%
Minibatch loss at step 4500: 0.757221
Minibatch accuracy: 88.3%
Validation accuracy: 85.5%
Minibatch loss at step 5000: 0.825938
Minibatch accuracy: 85.9%
Validation accurac

Validation accuracy: 88.4%

Test accuracy: 94.6%